## 3. Segy bin header

Binary Header - заголовок размера 400 байт, который содержит некоторое числовое описание SEG-Y файла и данных, хранящихся в нем. Бинарный заголовок состоит из именованных полей и их значений, подробное описание всех полей можно прочитать в SEG Technical Standards Committee SEG-Y revision 2.0 Data Exchange format (https://seg.org/Portals/0/SEG/News%20and%20Resources/Technical%20Standards/seg_y_rev2_0-mar2017.pdf)

Рассмотрим функционал библиотеки для работы с бинарным заголовком.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from timeit import default_timer as timer

In [2]:
import seismo_reader as sr

### 3.1 Описание класса 

Бинарный заголовок представлен классом sr.segy_bin_header. Описание методов класса можно получить по команде help:

In [3]:
help(sr.segy_bin_header)

Help on class segy_bin_header in module seismo_reader.seismo_reader:

class segy_bin_header(abstract_header)
 |  Method resolution order:
 |      segy_bin_header
 |      abstract_header
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: seismo_reader.seismo_reader.segy_bin_header) -> None
 |      
 |      2. __init__(self: seismo_reader.seismo_reader.segy_bin_header, fields_dict: Dict[str, Union[str, int, int, int, int, int, int, int, float, float, str]]) -> None
 |  
 |  raw_data(...)
 |      raw_data(self: seismo_reader.seismo_reader.segy_bin_header) -> List[int]
 |      
 |      Returns byte array in SEG-Y raw format
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from abstract_header:
 |  
 |  description(...)
 |      description(self: seismo_reader.seismo_reader.a

Как видно из помощи, класс является некоторым хранилищем значений вида __{'имя поля заголовка': значение}__ и поддерживает преобразование в/из __dict__.

Дополнительно представлены функции __get/set__, позволяющие манипулировать отдельными полями, что предпочтительнее в случае, когда необходимо обработать лишь небольшое число полей заголовка.

Доступен как пустой конструктор, так и конструктор из словаря. Есть возможность получить "сырые" 400 байт заголовка при помощи метода __raw_data__.

### 3.2 Чтение и просмотр бинарного заголовка из SEG-Y файла

Рассмотрим пример чтения бинарного заголовка из SEG-Y файла, опробуем функционал класса и приведем полученную информацию в _Userfriendly_ вид.

In [4]:
config = sr.reader_config()
config.filename = '2D.segy'
reader = sr.segy_reader(config)

Получить бинарный заголовок из открытого SEG-Y файла можно методом segy_reader.bin_header() (см. ?).

In [5]:
bh = reader.bin_header()

Преобразуем бинарный заголовок в словарь и посмотрим на его содержимое.

In [6]:
bh.to_dict()

{'Amplitude rec method': 0,
 'Aux traces count': 0,
 'Correlated traces': 0,
 'Data format': 8,
 'Endian': 0,
 'Ensemble fold': 1,
 'Extended aux traces count': 0,
 'Extended ensemble fold': 0,
 'Extended sample interval': 0.0,
 'Extended sample interval orig': 0.0,
 'Extended samples count': 0,
 'Extended samples count orig': 0,
 'Extended text headers count': 0,
 'Extended traces count': 0,
 'First trace offset': 0,
 'Gain recovered': 0,
 'Is same for file': 0,
 'Is segy 2': 0,
 'Job id': 1,
 'Line num': 1,
 'Max add trc headers count': 0,
 'Measurement system': 1,
 'Polarity code': 0,
 'Reel num': 1,
 'Sample interval': 2000.0,
 'Sample interval orig': 0.0,
 'Samples count': 1801,
 'Samples count orig': 0,
 'Signal polarity': 1,
 'Sorting code': 4,
 'Stream traces count': 0,
 'Sweep chanel trcs count': 0,
 'Sweep fr end': 0,
 'Sweep fr start': 0,
 'Sweep len': 0,
 'Sweep trc taper len end': 0,
 'Sweep trc taper len start': 0,
 'Sweep type': 0,
 'Taper type': 0,
 'Time basis': 0,
 'T

Как видно из содержимого, бинарный заголовок состоит из более чем сорока именованных полей.<br>
Некоторые поля имеют нулевое значение, это скорее всего означает, что такие поля не играют роли в интерпретации содержимого файла.

Наиболее важными полями являются:<br>
 >__Samples count__ - количество отсчетов в трассе,<br>
 >__Sample interval__ - временной (или глубинный) интервал между отсчетами в трассе<br>
 >__Data format__ - формат данных в трассе
 >__Endian__ - порядок байт при записи чисел в файле<br>

Значение по полю можно получить не только из словаря, но и методом get:

In [7]:
# По словарю
bh.to_dict()['Data format']

8

In [8]:
# Методом get
bh.get('Data format')

8

Имя поля можно вводить и в другом формате: без заглавной буквы и с '_' в качестве разделителя слов:

In [9]:
bh.get('data_format')

8

Так же класс предоставляет краткое описание каждого поля, которое можно получить с помощью метода __description__.<br>
Помимо самого описания предоставляются диапазоны байт для сопостовления с документацией.

Вывод всего словаря description довольно сложен для понимания, поэтому рассмотрим лишь описание поля __Data format__:

In [10]:
print(bh.description()['Data format'])

{'Bytes': '3225-3226', 'Description': "Data sample format code:\n\t1 - 4-byte IBM float\n\t2 - 4-byte, two's complement integer\n\t3 - 2-byte, two's complement integer\n\t4 - 4-byte fixed-point with gain (obsolete)\n\t5 - 4-byte IEEE float\n\t6 - 8-byte IEEE float\n\t7 - 3-byte two's complement integer\n\t8 - 1-byte, two's complement integer\n\t9 - 8-byte, two's complement integer\n\t10 - 4-byte, unsigned integer\n\t11 - 2-byte, unsigned integer\n\t12 - 8-byte, unsigned integer\n\t15 - 3-byte, unsigned integer\n\t16 - 1-byte, unsigned integer"}


Отдельно выведем диапазон байт и описание для поля __Data format__:

In [11]:
# Диапазон байт
print(bh.description()['Data format']['Bytes'])

3225-3226


In [12]:
# Само описание
print(bh.description()['Data format']['Description'])

Data sample format code:
	1 - 4-byte IBM float
	2 - 4-byte, two's complement integer
	3 - 2-byte, two's complement integer
	4 - 4-byte fixed-point with gain (obsolete)
	5 - 4-byte IEEE float
	6 - 8-byte IEEE float
	7 - 3-byte two's complement integer
	8 - 1-byte, two's complement integer
	9 - 8-byte, two's complement integer
	10 - 4-byte, unsigned integer
	11 - 2-byte, unsigned integer
	12 - 8-byte, unsigned integer
	15 - 3-byte, unsigned integer
	16 - 1-byte, unsigned integer


Таким образом, в представленном файле отсчеты трасс хранятся в формате __1-byte, two's complement integer__.

Приведем полученную информацию в более красивый вид для вывода пользователю-интерпретатору.

Для этого сделаем следующее:<br>
>1. Разделим полученный словарь полей и значений бинарного заголовка на 2 части: ненулевую и нулевую.
>2. Отсортируем отдельно обе части по диапазону байт (для более удобного сопоставления с документацией).
>3. Добавим к каждому поля описание и выведем все это на экран.

Работа со словарями в таких операциях довольно сложна, на помощь приходит мощный инструмент pandas.

In [13]:
# Сохраним словарь в переменную d
d = bh.to_dict()

In [14]:
# Заполним отдельно 2 DataFrame'а, один со значениями полей, второй с описанием
df_values = pd.DataFrame({'Value' : list(d.values())}, index=list(d.keys()))
df_descr = pd.DataFrame.from_dict(bh.description()).T

# Сольем в единый DataFrame
df = pd.concat([df_values, df_descr], axis=1)

# Отсортируем ненулевую и нулевую часть по диапазонам байт
df = pd.concat([
    df[df['Value'] > 0].sort_values(by=['Bytes'], ascending=True),
    df[df['Value'] <= 0].sort_values(by=['Bytes'], ascending=True)],
    axis=0
)

In [15]:
df

,Value,Bytes,Description
Job id,1.0,3201-3204,Job identification number
Line num,1.0,3205-3208,"Line number. For 3D data, this will contain th..."
Reel num,1.0,3209-3212,Reel number
Traces count,1.0,3213-3214,Number of data traces per ensemble
Sample interval,2000.0,3217-3218,Sample interval (us / Hz / m / ft)
Samples count,1801.0,3221-3222,Number of samples per data trace
Data format,8.0,3225-3226,Data sample format code:\n\t1 - 4-byte IBM flo...
Ensemble fold,1.0,3227-3228,The expected number of data traces per trace e...
Sorting code,4.0,3229-3230,Trace sorting code:\n\t-1 - Other\n\t0 - Unkno...
Vert sum code,1.0,3231-3232,Vertical sum code:\n\t1 - No sum\n\t2 - two su...


В таком виде уже не стыдно показывать заголовок профессиональному интерпретатору.

Однако, использовать pandas.DataFrame для хранения заголовка не рекомендуется. DataFrame хранит значения полей в едином типе (np.float), это может привести к дальнейшей потери данных, да и сам объект класса segy_bin_header не позволит устанавливать некоторые целочисленные поля при помощи числа с плавающей точкой.

Рекомендуется манипулировать заголовком через pandas, но сохранять заголовок через сереализацию класса segy_bin_header, либо как словарь.

Обработка ошибок класса segy_bin_header осуществляется посредством исключений.

Попытаемся получить несуществующее поле заголовка.

In [16]:
bh.get('aaa')

ValueError: segy_bin_header.cpp:214: invalid field name

Получаем исключение __ValueError__ с описанием _invalid field name_.

Так же можно посмотреть (или манипулировать) с "сырым" заголовком через метод __raw_data__:

In [17]:
rbh = np.array(bh.raw_data(), dtype=np.int8) #int8 == byte

rbh

array([  0,   0,   0,   1,   0,   0,   0,   1,   0,   0,   0,   1,   0,
         1,   0,   0,   7, -48,   0,   0,   7,   9,   0,   0,   0,   8,
         0,   1,   0,   4,   0,   1,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   1,   0,   1,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

Размер такого массива, как и ожидалось, 400 байт:

In [18]:
rbh.shape

(400,)

### 3.3 Изменение бинарного заголовка

Для изменения содержимого бинарного заголовка предоставляются методы __set/from_dict__ и конструктор __init__(self, dict). 

Изменять бинарный заголовок, сопряженный со своим файлом, запрещено, так как настройки чтения файла (такие как data_format, endian) можно менять через класс segy_reader, а остальные поля в заголовке предоставляются для ознакомления с файлом.

Действительно, попробовав изменить полученный из файла заголовок, получаем исключение (object is read only):

In [19]:
# Через словарь:
bh.from_dict(bh.to_dict())

RuntimeError: segy_bin_header.cpp:219: object is read only

In [20]:
# Через метод set
bh.set('Data format', 15)

RuntimeError: segy_bin_header.cpp:219: object is read only

Если в бинарном заголовке есть ошибка, либо необходимо задать свои значения каки-либо полей, необходимо создать новый бинарный заголовок и далее работать с ним.

Рассмотрим пример создания нового бинарного заголовка на основе bh с изменением некоторых полей.

Создадим пустой бинарный заголовок

In [21]:
bh_new = sr.segy_bin_header()

Выведем его поля в виде словаря, убедимся, что он пустой. Так рассмотрим "сырое" представление:

In [22]:
np.array(list(bh_new.to_dict().values()))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [23]:
np.array(bh_new.raw_data(), dtype=np.int8)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Мы видим, что у пустого бинарного заголовка "сырое" представление ненулевое. На позиции 92 (байты 92-93) стоят какие-то числа. Эти числа показывают, в каком порядке байт (endian) будет записан файл (вероятнее всего, сейчас там записан маркер для порядка big-endian). Даже при пустом заголовке класс segy_bin_header оставляет эту информацию. Все остальные байты нулевые.

Заполним его поля, используя существующий заголовок bh и выведем новый заголовок в виде словаря:

In [24]:
bh_new.from_dict(bh.to_dict())

In [25]:
bh_new.to_dict()

{'Amplitude rec method': 0,
 'Aux traces count': 0,
 'Correlated traces': 0,
 'Data format': 8,
 'Endian': 0,
 'Ensemble fold': 1,
 'Extended aux traces count': 0,
 'Extended ensemble fold': 0,
 'Extended sample interval': 0.0,
 'Extended sample interval orig': 0.0,
 'Extended samples count': 0,
 'Extended samples count orig': 0,
 'Extended text headers count': 0,
 'Extended traces count': 0,
 'First trace offset': 0,
 'Gain recovered': 0,
 'Is same for file': 0,
 'Is segy 2': 0,
 'Job id': 1,
 'Line num': 1,
 'Max add trc headers count': 0,
 'Measurement system': 1,
 'Polarity code': 0,
 'Reel num': 1,
 'Sample interval': 2000.0,
 'Sample interval orig': 0.0,
 'Samples count': 1801,
 'Samples count orig': 0,
 'Signal polarity': 1,
 'Sorting code': 4,
 'Stream traces count': 0,
 'Sweep chanel trcs count': 0,
 'Sweep fr end': 0,
 'Sweep fr start': 0,
 'Sweep len': 0,
 'Sweep trc taper len end': 0,
 'Sweep trc taper len start': 0,
 'Sweep type': 0,
 'Taper type': 0,
 'Time basis': 0,
 'T

Визуально словари старого и нового заголовка идентичны. Проверка на равенство так же проходит:

In [26]:
bh_new.to_dict() == bh.to_dict()

True

Предположим, после обработки, мы изменили количество отсчетов в трассах, интервал отсчетов (применили какую-либо интерполяцию). Запишем новые значения этих полей в заголовок.

Это можно сделать как через словарь, так и через метод __set__:

In [27]:
# Через словарь:
d = bh_new.to_dict()
d.update({'Samples count': 500, 'Sample interval' : 2600})

bh_new.from_dict(d)

# Проверим изменения полей
print(bh_new.to_dict()['Samples count'])
print(bh_new.to_dict()['Sample interval'])

500
2600.0


In [28]:
# Через метод set:
bh_new.set('Samples count', 1000)
bh_new.set('Sample interval', 2000)

# Проверим изменения полей
print(bh_new.to_dict()['Samples count'])
print(bh_new.to_dict()['Sample interval'])

1000
2000.0


Некоторые поля требуют только целочисленного значения. Так, например, попробовав изменить поле Data format в формате цисла с плавающей запятой, получаем исключение:

In [29]:
bh_new.set('Data format', 2.0)

ValueError: data_types.h:69: Mixing inappropriate types (float with enum segy_data_format)

Действительно, некоторые поля класса segy_bin_header не просто целочисленные, а имеют тип enum. Установить их можно через целочисленное значение, или передав enum напрямую (рекомендуется второй вариант):

In [30]:
# Целое число - OK
bh_new.set('Data format', 2)

In [31]:
# Enum число - OK
bh_new.set('Data format', sr.seismic_data_type.FLOAT)

### 3.4 Замечания 

Изменять поля __Data format__ и __Endian__ при записи SEG-Y файла не имеет смысла, эти поля будут автоматически перезаписаны исходя из конфигурации объекста segy_writer